# Trading Places
1. Read in data using Dask __(1 point)__
2. Descriptive Statistics: __(2 points)__
    
    a. In the year 2012, who were the top 10 countries with the most trading partners? What about the bottom 10?
    
    b. Using the designation [here]( https://www.foreign-trade.com/reference/hscode.htm), describe the trade volume of the whole dataset in terms of value and list the five highest value sectors in the entire sample period.

3. Country statistics: __(3 points)__

    a. Calculate the top 10 exports (in terms of value) of our chosen country in 2012.

    b. Calculate the top 10 goods with the highest trade volume for both exports and imports in terms of:
    
        i. Value
    
        ii. Quantity

3. The Gravity Equation Relationship __(4 points)__

    a. Using the country shapefile, calculate the distance in kilometers between the centroid of our chosen country and all the other centroids. You may use either haversine distance with the geographic CRS or use a projected CRS and use the `distance` function of `geopandas`.
    
    b. Using the country we chose in class, create a scatterplot of distance and export volume (quantity and value) in logarithm form.
    
    c. Calculate the correlations between exports and distance for both value and quantity. Interpret the correlations for each of these factors.
    
    d. Using the designation you created in 2.d., for which sectors are exports (quantity) most strongly related to distance for this country?
    
4. Estimation __(6 points)__
    
    a. Create a function called "SSE_1p" that does the following given a scalar input $\beta$:
        1. Subtracts the logarithm of quantity exports from the logarithm of distance times $\beta$.
$$ e = log(E_{ij}) - \beta log(d_{ij}) $$
        2. Squares the error.
        3. Returns the sum of the squared error. 
    b. Plot the function from -1 to +3 with steps of .005, and then find the minimum of the function with the numpy function "argmin." Is it what you expected?
    
    c. Create another function called "SSE_2p" does the same as "SSE_1p" but instead takes a __vector__ as an argument:
 $$ e = log(E_{ij}) - \beta[0] - \beta[1] log(d_{ij})$$
 
     d. Do a grid search with the first parameter range as 1500 to 2500 in steps of 10, and the second parameter range as -1 to +3 in steps of .005 as before. Do it using either `dask` or `multiprocessing`. Comment on how or why the parameter on $log(d_{ij})$ is different than what you found in part 4b.
     
     e. Now check your results using the "econtools" package "reg" function by running OLS with a constant term.
     
     f. Finally, merge in the GDP numbers from the World Bank and do the regression again, except this time the log of each country's GDP as covariates. How did the coefficient on distance change?
     
5. An analyst asks you "for which commodity sectors is distance most important in terms of exporting?" Describe what your economic intuition would say about the answer, and then also map out an analysis strategy for answering their question. As usual, describe: __(4 points)__
    - What data you would need.
    - How you would manipulate the data.
    - What analysis you would run.

In [58]:
import glob
import pandas as pd
import numpy as np
import dask.dataframe as dd
import matplotlib.pyplot as plt
import json
import dask
import geopandas as gp
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt


def haversine(row):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    lon1, lat1, lon2, lat2 = row
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [59]:
#Each row represents the trade flows from country $i$ to country $j$ for product $k$ in time $t$ 
#in terms of either value or quantity.

In [60]:
products = pd.read_csv("/Users/isaacwerries/Desktop/test/Hw3/product_codes_HS12_V202001.csv")
products = products.rename(columns={"code":"k"})
products

,k,description
0,10121,"Horses: live, pure-bred breeding animals"
1,10129,"Horses: live, other than pure-bred breeding an..."
2,10130,Asses: live
3,10190,Mules and hinnies: live
4,10221,"Cattle: live, pure-bred breeding animals"
...,...,...
5197,970300,"Sculptures and statuary: original, in any mate..."
5198,970400,"Stamps, postage or revenue: stamp-postmarks, f..."
5199,970500,Collections and collectors' pieces: of zoologi...
5200,970600,Antiques: of an age exceeding one hundred years


In [61]:
df = pd.read_csv('/Users/isaacwerries/Desktop/test/Hw3/BACI_HS12_Y2012_V202001.csv.crdownload')
df

,t,i,j,k,v,q
0,2012,4,8,610469,2.734000,0.044
1,2012,4,12,91099,1.185000,1.200
2,2012,4,32,392630,1.274453,0.123
3,2012,4,32,730729,2.018000,0.100
4,2012,4,32,853400,5.530000,0.028
...,...,...,...,...,...,...
6936986,2012,894,854,841459,24.335000,1.000
6936987,2012,894,854,843041,770.661000,18.000
6936988,2012,894,854,846799,29.269000,0.343
6936989,2012,894,854,848140,10.207000,1.341


In [62]:
df12 = dd.read_csv('/Users/isaacwerries/Desktop/test/Hw3/BACI_HS12_Y*.csv.crdownload')
df12

,t,i,j,k,v,q
npartitions=37,,,,,,
,int64,int64,int64,int64,float64,float64
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [63]:
df12_names = df12.merge(products)

In [64]:
countries = pd.read_csv("/Users/isaacwerries/Desktop/test/Hw3/country_codes_V202001.csv",encoding = "ISO-8859-1")
countries = countries.rename(columns={"country_code":"i","country_name_full":"name_i"})
df12_names = df12_names.merge(countries[['i','name_i']])

countries = pd.read_csv("/Users/isaacwerries/Desktop/test/Hw3/country_codes_V202001.csv",encoding = "ISO-8859-1")
countries = countries.rename(columns={"country_code":"j","country_name_full":"name_j"})
df12_names = df12_names.merge(countries[['j','name_j']])
df12_names.head()

,t,i,j,k,v,q,description,name_i,name_j
0,2012,4,8,610469,2.734000,0.044000,"Trousers, bib and brace overalls, breeches and...",Afghanistan,Albania
1,2012,32,8,170490,22.438000,8.443000,"Sugar confectionery: (excluding chewing gum, i...",Argentina,Albania
2,2012,32,8,300490,166.408111,3.360398,Medicaments: consisting of mixed or unmixed pr...,Argentina,Albania
3,2012,32,8,121190,33.283929,2.216000,Plants and parts (including seeds and fruits) ...,Argentina,Albania
4,2012,32,8,870829,1.466350,0.130000,"Vehicles: parts and accessories, of bodies, ot...",Argentina,Albania


In [65]:
df12_names.loc[df12_names.t==2012].groupby("name_i")['name_j'].nunique().nlargest(10).compute()

name_i
USA, Puerto Rico and US Virgin Islands    214
Netherlands                               213
Belgium-Luxembourg                        212
Germany                                   212
United Kingdom                            212
Republic of Korea                         211
Italy                                     210
Switzerland, Liechtenstein                210
France, Monaco                            209
Canada                                    208
Name: name_j, dtype: int64

In [66]:
df12_names.loc[df12_names.t==2012].groupby("name_i")['name_j'].nunique().nsmallest(10).compute()

name_i
Bonaire, Saint Eustatius and Saba      1
Saint Maarten (Dutch part)             4
South Sudan                            9
Saint Pierre and Miquelon             14
Palau                                 16
Norfolk Islands                       18
Pitcairn                              19
Christmas Islands                     22
French South Antarctic Territories    23
Northern Mariana Islands              23
Name: name_j, dtype: int64

In [90]:
with open("/Users/isaacwerries/Desktop/test/sector_list.json") as f:
    sectors = json.load(f)

In [91]:
df12_names['sector'] = df12_names.k.astype(str).str[:-4]
df12_names = df12_names.replace({'sector':sectors})
df12_names.head(15)

,t,i,j,k,v,q,description,name_i,name_j,sector
0,2012,4,8,610469,2.734000,0.044000,"Trousers, bib and brace overalls, breeches and...",Afghanistan,Albania,Textiles
1,2012,32,8,170490,22.438000,8.443000,"Sugar confectionery: (excluding chewing gum, i...",Argentina,Albania,Foodstuffs
2,2012,32,8,300490,166.408111,3.360398,Medicaments: consisting of mixed or unmixed pr...,Argentina,Albania,Chemicals & Allied Industries
3,2012,32,8,121190,33.283929,2.216000,Plants and parts (including seeds and fruits) ...,Argentina,Albania,Vegetable Products
4,2012,32,8,870829,1.466350,0.130000,"Vehicles: parts and accessories, of bodies, ot...",Argentina,Albania,Transportation
5,2012,32,8,190590,25.786622,14.737897,Food preparations: bakers' wares n.e.c. in hea...,Argentina,Albania,Foodstuffs
6,2012,32,8,30617,71.111252,22.009943,"Crustaceans: frozen, shrimps and prawns, exclu...",Argentina,Albania,Animal & Animal Products
7,2012,32,8,80550,666.867463,874.006382,"Fruit, edible: lemons (Citrus limon, Citrus li...",Argentina,Albania,Vegetable Products
8,2012,32,8,80830,1.124994,1.440000,"Fruit, edible: pears, fresh",Argentina,Albania,Vegetable Products
9,2012,32,8,630900,2.480650,0.400000,"Clothing: worn, and other worn articles",Argentina,Albania,Textiles


In [11]:
brazil = df12_names.loc[(df12_names.t==2012) & (df12_names.name_i=="Brazil")]
brazil

,t,i,j,k,v,q,description,name_i,name_j
npartitions=37,,,,,,,,,
,int64,int64,int64,int64,float64,float64,object,object,object
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [12]:
brazil[['v','description']].nlargest(10,columns = 'v').compute()

,v,description
768321,1.417604e+07,Iron ores and concentrates: non-agglomerated
769050,1.195270e+07,"Soya beans: other than seed, whether or not br..."
153857,5.998622e+06,Oils: petroleum oils and oils obtained from bi...
768427,4.810871e+06,Oils: petroleum oils and oils obtained from bi...
1058803,3.431956e+06,Oils: petroleum oils and oils obtained from bi...
563382,2.613074e+06,Iron ores and concentrates: non-agglomerated
41311,2.198287e+06,Vehicles: spark-ignition internal combustion r...
387397,1.894189e+06,Oil-cake and other solid residues: whether or ...
387165,1.624865e+06,Petroleum oils and oils from bituminous minera...
153325,1.566110e+06,Iron or non-alloy steel: semi-finished product...


In [13]:
df12_names.groupby('description')['v'].sum().nlargest(10).compute()

description
Oils: petroleum oils and oils obtained from bituminous minerals, crude                                                                                                                                                                      6.701110e+09
Petroleum oils and oils from bituminous minerals, not crude: preparations n.e.c. containing by weight 70% or more of petroleum oils or oils from bituminous minerals: these being the basic constituents of the preparations: waste oils    4.176180e+09
Vehicles: spark-ignition internal combustion reciprocating piston engine, cylinder capacity exceeding 1500cc but not exceeding 3000cc                                                                                                       1.710643e+09
Telephones for cellular networks or for other wireless networks                                                                                                                                                                             1.642

In [14]:
df12_names.groupby('description')['q'].sum().nlargest(10).compute()

description
Oils: petroleum oils and oils obtained from bituminous minerals, crude                                                                                                                                                                      1.235678e+10
Iron ores and concentrates: non-agglomerated                                                                                                                                                                                                8.703451e+09
Petroleum oils and oils from bituminous minerals, not crude: preparations n.e.c. containing by weight 70% or more of petroleum oils or oils from bituminous minerals: these being the basic constituents of the preparations: waste oils    6.320674e+09
Coal: bituminous, whether or not pulverised, but not agglomerated                                                                                                                                                                           6.280

In [37]:
countries = gp.read_file('countries.geojson')

DriverError: countries.geojson: No such file or directory